# DS 4. 미분, 경사하강법, grid search

> "작성완료"

- toc:true
- branch: master
- badges: true
- comments: true
- [python, Data Science]

---
# Data Science
- lenture: Data Science_4-1nd week of lectures.
- lenture date: 2022-03-23
- lecturer: Guebin choi
- study date: 2022-03-23
- author: Kione kim
---

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 

In [3]:
import tensorflow.experimental.numpy as tnp   # tnp를 사용하면 넘파이에 익숙한 문법을 모두 쓸 수 있음
tnp.experimental_enable_numpy_behavior()   # 기존에 생성된 tf.constant 자료형은 넘파이와 유사하게 동작한다. 

## 미분

### tf.GradientTape()

#### 카페예제

`-` $x,y$ 선언

In [5]:
x=tnp.array([20.1,22.2,22.7,23.3,24.4,25.1,26.2,27.3,28.4,30.4])
x

<tf.Tensor: shape=(10,), dtype=float64, numpy=array([20.1, 22.2, 22.7, 23.3, 24.4, 25.1, 26.2, 27.3, 28.4, 30.4])>

In [7]:
tf.random.set_seed(50000)
y=10.2+ 2.2*x+ tnp.random.randn(10)
y

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([53.12550628, 59.48221878, 61.86480622, 64.06900254, 63.52340823,
       62.85870759, 67.29683042, 69.54750897, 72.283444  , 76.08682149])>

`-` 임의의 $β_0, β_1$ 값 설정

In [10]:
beta0=tf.Variable([9.0])
beta1=tf.Variable([2.0])

`-` with문

In [13]:
with tf.GradientTape(persistent=True) as mytape:
    loss=sum((y-beta0-beta1*x)**2)
mytape.gradient(loss,beta0), mytape.gradient(loss,beta1)

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-119.87651], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-3008.6094], dtype=float32)>)

- loss(잔차제곱합)를 β_0, β_1 각각에 대해 미분하여 대입한 값을 보았더니 0과는 거리가 멀다. 더 나은 직선이 많을 것 같다.

- 다른 베타값에 대한 미분값을 살펴보고 싶은데 위의 코드처럼 각각을 적용하는 것은 확장성이 떨어진다.

- 매트릭스 미분을 활용해보자!

`-` 매트릭스를 활용한 계산

---

#### 복습

`-` 모형의 매트릭스화

`-` 우리의 모형

$y_i = \beta_0 + \beta_1 x_i + \epsilon_i, \quad i=1,2,\dots,10$ 

이를 풀어서 쓰면
    
$\begin{cases}
y_1 = \beta_0 +\beta_1 x_1 + \epsilon_1 \\ 
y_2 = \beta_0 +\beta_1 x_2 + \epsilon_2 \\ 
\dots \\ 
y_{10} = \beta_0 +\beta_1 x_{10} + \epsilon_{10} 
\end{cases}$

아래와 같다.

$\begin{bmatrix} 
y_1 \\ 
y_2 \\ 
\dots \\
y_{10} 
\end{bmatrix} 
= \begin{bmatrix} 
1 & x_1 \\ 
1 & x_2 \\ 
\dots & \dots \\
1 & x_{10} 
\end{bmatrix}\begin{bmatrix}\beta_0 \\ \beta_1 \end{bmatrix} + \begin{bmatrix} 
\epsilon_1 \\ 
\epsilon_2 \\ 
\dots \\
\epsilon_{10} 
\end{bmatrix} $

벡터와 매트릭스 형태로 표현하면

${\bf y} = {\bf X} {\boldsymbol \beta} + \boldsymbol{\epsilon}$

`-` 손실함수의 매트릭스화

$loss=\sum_{i=1}^{n}(y_i-\beta_0-\beta_1x_i)^2$

이를 벡터로 표현하면,

$loss=({\bf y}-{\bf X}{\boldsymbol \beta})^\top({\bf y}-{\bf X}{\boldsymbol \beta})={\bf y}^\top {\bf y} - {\bf y}^\top {\bf X}{\boldsymbol\beta} - {\boldsymbol\beta}^\top {\bf X}^\top {\bf y} + {\boldsymbol\beta}^\top {\bf X}^\top {\bf X} {\boldsymbol\beta}$

`-` 미분하는 과정의 매트릭스화 

loss를 최소화하는 ${\boldsymbol \beta}$를 구해야하므로 loss를 ${\boldsymbol \beta}$로 미분한식을 0이라고 놓고 풀면 된다. 

$\frac{\partial}{\partial \boldsymbol{\beta}} loss = \frac{\partial}{\partial \boldsymbol{\beta}} {\bf y}^\top {\bf y} - \frac{\partial}{\partial \boldsymbol{\beta}} {\bf y}^\top {\bf X}{\boldsymbol\beta} - \frac{\partial}{\partial \boldsymbol{\beta}} {\boldsymbol\beta}^\top {\bf X}^\top {\bf y} + \frac{\partial}{\partial \boldsymbol{\beta}} {\boldsymbol\beta}^\top {\bf X}^\top {\bf X} {\boldsymbol\beta}$ 

$= 0 - {\bf X}^\top {\bf y}- {\bf X}^\top {\bf y} + 2{\bf X}^\top {\bf X}{\boldsymbol\beta} $

따라서 $\frac{\partial}{\partial \boldsymbol{\beta}}loss=0$을 풀면 아래와 같다. 

$\boldsymbol{\hat\beta}= ({\bf X}^\top {\bf X})^{-1}{\bf X}^\top {\bf y} $

---

`-` 매트릭스로 $x,y$ 선언

In [21]:
[1]*10 + [2]*10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [46]:
tnp.array([1]*10 + [2]*10).reshape(2,10).T

<tf.Tensor: shape=(10, 2), dtype=int32, numpy=
array([[1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2]])>

In [59]:
x=tnp.array([1]*10 + [20.1,22.2,22.7,23.3,24.4,25.1,26.2,27.3,28.4,30.4]).reshape(2,10).T
x

<tf.Tensor: shape=(10, 2), dtype=float64, numpy=
array([[ 1. , 20.1],
       [ 1. , 22.2],
       [ 1. , 22.7],
       [ 1. , 23.3],
       [ 1. , 24.4],
       [ 1. , 25.1],
       [ 1. , 26.2],
       [ 1. , 27.3],
       [ 1. , 28.4],
       [ 1. , 30.4]])>

In [71]:
beta_true= tnp.array([[10.2,2.2]])
beta_true

<tf.Tensor: shape=(1, 2), dtype=float64, numpy=array([[10.2,  2.2]])>

In [72]:
x@beta_true

ValueError: Matrix size-incompatible: In[0]: [10,2], In[1]: [1,2] [Op:MatMul]

- 메트릭스 선언을 잘못하면 오류가 남..! 잘못된 계산

In [73]:
beta_true= tnp.array([[10.2],[2.2]])
beta_true

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[10.2],
       [ 2.2]])>

- matrix 선언이 중요함,,

In [75]:
x@beta_true

<tf.Tensor: shape=(10, 1), dtype=float64, numpy=
array([[54.42],
       [59.04],
       [60.14],
       [61.46],
       [63.88],
       [65.42],
       [67.84],
       [70.26],
       [72.68],
       [77.08]])>

In [78]:
tnp.random.seed(50000)
y=x@beta_true + tnp.random.randn(10).reshape(10,1)
y

<tf.Tensor: shape=(10, 1), dtype=float64, numpy=
array([[53.12550628],
       [59.48221878],
       [61.86480622],
       [64.06900254],
       [63.52340823],
       [62.85870759],
       [67.29683042],
       [69.54750897],
       [72.283444  ],
       [76.08682149]])>

`-` 미분하고 싶은 $β$값 정의

In [81]:
beta= tnp.array([[9.0],[2.0]])
beta

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[9.],
       [2.]])>

`-` with문

In [82]:
with tf.GradientTape() as mytape:
    loss=(y-x@beta).T @ (y-x@beta)

In [83]:
mytape.gradient(loss,beta)

- β가 tf.constant로 정의되었기 때문에 계산결과 Nan이 되었다. 

In [84]:
with tf.GradientTape() as mytape:
    mytape.watch(beta)
    loss=(y-x@beta).T @ (y-x@beta)

In [85]:
mytape.gradient(loss,beta)

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[ -119.87650904],
       [-3008.60929929]])>

- 앞서 직접 구한 값과 동일하게 나왔다

`-` 다음과 같이 변환하여 보다 직관적으로 표현할 수 있다.

In [95]:
beta= tnp.array([[9.0],[2.0]])

with tf.GradientTape(persistent=True) as mytape:
    mytape.watch(beta)
    yhat=x@beta # yhat- 추정치
    loss=(y-yhat).T @ (y-yhat)

In [96]:
mytape.gradient(loss,beta)

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[ -119.87650904],
       [-3008.60929929]])>

``` python
with tf.GradientTape() as mytape:
    mytape.watch(beta)
    yhat=x@beta # yhat- 추정치
    loss=(y-yhat).T @ (y-yhat)
    ...
    ...
    ...
    등등
    
: 계속해서 선형변환, 비선형변환 등등을 추가하여 계산할 수 있어 매우 편리하다고 한다...
```

`-` 이론적 풀이

$loss'(\beta)= - {2\bf X}' {\bf y} + 2{\bf X}' {\bf X}{\boldsymbol\beta} $

In [99]:
-2 * x.T @ y + 2 * x.T @ x @ beta

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[ -119.87650904],
       [-3008.60929929]])>

`-` 이론적인 $β$의 최적값을 찾아보고 (즉 $\hat\beta$을 찾고) 그 지점에서 loss의 미분값(=접선의 기울기)을 구하라. 결과가 0인지 확인하라. 단 0은 길이가 2이고 각 원소가 0인 벡터이다.

$\boldsymbol{\hat\beta}= ({\bf X}^\top {\bf X})^{-1}{\bf X}^\top {\bf y} $

In [103]:
bhat = tf.linalg.inv(x.T @ x) @ x.T @ y # tf.linalg.inv()는 역행렬 구하는 기능
bhat

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[13.49949892],
       [ 2.05974916]])>

- 앞서 구한 것보다 매우 근접한 값이 나왔다.

In [104]:
with tf.GradientTape(persistent=True) as mytape:
    mytape.watch(bhat)
    yhat= x@bhat
    loss= (y-yhat).T @ (y-yhat) 

In [105]:
mytape.gradient(loss,bhat)

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[-1.87583282e-12],
       [-4.64863703e-11]])>

- bhat에서 접선의 기울기는 0과 매우 가까운 값이 계산되었다.

## 경사하강법

### 최적화문제

`-` $loss= (\frac{1}{2}\beta -1)^2$를 최소로 하는 $\beta$를 컴퓨터를 통해 구하는 문제를 생각해보자. (답은 당연히 $\beta=2$)

#### 방법 1: grid search

``` python
알고리즘:
    step1: beta를 적당한 범위 내 값들로 설정한다.
    step2: 각각의 범위 내 값들에 대한 loss를 계산한다.
    step3: loss값들 중 가장 작은 beta 값을 찾는다.        
```

In [110]:
beta= tnp.linspace(-10,10,100)
loss= (beta/2 -1)**2

In [111]:
loss

<tf.Tensor: shape=(100,), dtype=float64, numpy=
array([3.60000000e+01, 3.47980818e+01, 3.36165697e+01, 3.24554637e+01,
       3.13147638e+01, 3.01944700e+01, 2.90945822e+01, 2.80151005e+01,
       2.69560249e+01, 2.59173554e+01, 2.48990919e+01, 2.39012346e+01,
       2.29237833e+01, 2.19667381e+01, 2.10300990e+01, 2.01138659e+01,
       1.92180390e+01, 1.83426181e+01, 1.74876033e+01, 1.66529946e+01,
       1.58387920e+01, 1.50449954e+01, 1.42716049e+01, 1.35186205e+01,
       1.27860422e+01, 1.20738700e+01, 1.13821039e+01, 1.07107438e+01,
       1.00597898e+01, 9.42924191e+00, 8.81910009e+00, 8.22936435e+00,
       7.66003469e+00, 7.11111111e+00, 6.58259361e+00, 6.07448220e+00,
       5.58677686e+00, 5.11947760e+00, 4.67258443e+00, 4.24609734e+00,
       3.84001632e+00, 3.45434139e+00, 3.08907254e+00, 2.74420977e+00,
       2.41975309e+00, 2.11570248e+00, 1.83205795e+00, 1.56881951e+00,
       1.32598714e+00, 1.10356086e+00, 9.01540659e-01, 7.19926538e-01,
       5.58718498e-01, 4.1791

---

`-` 가장 작은 값을 찾는 함수 `tf.argmin()`이용

In [115]:
tf.argmin([1,-1,2,-2])

<tf.Tensor: shape=(), dtype=int64, numpy=3>

- 가장 작은 값이 있는 인덱스를 출력해준다

---

In [112]:
tf.argmin(loss)

<tf.Tensor: shape=(), dtype=int64, numpy=59>

In [114]:
beta[tf.argmin(loss)]

<tf.Tensor: shape=(), dtype=float64, numpy=1.9191919191919187>

In [117]:
beta[59]

<tf.Tensor: shape=(), dtype=float64, numpy=1.9191919191919187>

- 이론적 값인 2와 근접한 값이 나왔음

In [118]:
beta[60]

<tf.Tensor: shape=(), dtype=float64, numpy=2.121212121212121>

In [119]:
beta[58]

<tf.Tensor: shape=(), dtype=float64, numpy=1.7171717171717162>

In [122]:
loss[59], loss[60], loss[58]

(<tf.Tensor: shape=(), dtype=float64, numpy=0.0016324864809713505>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.0036730945821854847>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.01999795939189892>)

- 앞 뒤로 한 개씩 차이나는 인덱스에 대한 loss값을 계산한 결과를 보니, 59번째 인덱스의 loss가 가장 작은 것을 알 수 있다.

##### 그리드서치의 문제점

`-` 문제1: local minimum

- 지정한 범위 외에서 더 작은 loss값이 나올 수 있음. 즉, 지정한 범위 내에 global minimum이 존재한다는 보장이 없음

`-` 문제2: 효율적이지 않음

- 모든 loss를 찾는 건 비효율적일 수 있음. 초반 또는 중간에 $loss=0$인 값을 찾았음에도 계산을 멈추지 않고 (더 이상 탐색할 필요가 없음에도) 끝까지 계산함

#### 추가학습

`-` 그리드서치를 이용하여 $(x-1)^2$을 최소화하는 $x$값을 구하여라. 

In [139]:
beta= tnp.linspace(-5,5,1000)
loss= (beta-1)**2

In [140]:
tf.argmin(loss)

<tf.Tensor: shape=(), dtype=int64, numpy=599>

In [143]:
beta[599]

<tf.Tensor: shape=(), dtype=float64, numpy=0.9959959959959956>

- 이론적인 값인 1과 근접한 값이 나왔다